### Pandas trick #1: Keep rows that contain smallest values 

In [1]:
import pandas as pd

df = pd.DataFrame({
    "A": ["foo", "foo", "foo", "foo", "foo", "bar", "bar", "bar", "bar"],
    "B": ["one", "one", "one", "two", "two", "one", "one", "two", "two"],
    "C": ["small", "large", "large", "small", "small", "large", "small", "small", "large"],
    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]
})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [2]:
df.groupby("A")["E"].transform("min")

0    2
1    2
2    2
3    2
4    2
5    6
6    6
7    6
8    6
Name: E, dtype: int64

In [3]:
df.groupby("A")["E"].transform("min").eq(df["E"])

0     True
1    False
2    False
3    False
4    False
5     True
6    False
7    False
8    False
Name: E, dtype: bool

In [4]:
df.loc[df.groupby("A")["E"].transform("min").eq(df["E"])]

,A,B,C,D,E
0,foo,one,small,1,2
5,bar,one,large,4,6


#### add a column with group summaries

In [5]:
df["group_count"] = df.groupby("A")["E"].transform("count")  # or size
df["group_min"] = df.groupby("A")["E"].transform("min")
df["group_mean"] = df.groupby("A")["E"].transform("mean")
df

,A,B,C,D,E,group_count,group_min,group_mean
0,foo,one,small,1,2,5,2,4.4
1,foo,one,large,2,4,5,2,4.4
2,foo,one,large,2,5,5,2,4.4
3,foo,two,small,3,5,5,2,4.4
4,foo,two,small,3,6,5,2,4.4
5,bar,one,large,4,6,4,6,8.0
6,bar,one,small,5,8,4,6,8.0
7,bar,two,small,6,9,4,6,8.0
8,bar,two,large,7,9,4,6,8.0


### Pandas trick #2: named aggregation

In [6]:
import pandas as pd
import numpy as np

animals = pd.DataFrame({
    "kind": ["cat", "dog", "cat", "dog"],
    "height": [9.1, 6.0, 9.5, 34.0],
    "weight": [7.9, 7.5, 9.9, 198.0]
})
animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [7]:
animals.groupby("kind", as_index=False).agg(
    min_height=("height", "min"),
    max_height=("height", "max"),
    average_weight=("weight", np.mean)
)

,kind,min_height,max_height,average_weight
0,cat,9.1,9.5,8.90
1,dog,6.0,34.0,102.75


In [8]:
animals.groupby("kind", as_index=False).agg({"height": "min", "weight": "mean"})

,kind,height,weight
0,cat,9.1,8.90
1,dog,6.0,102.75


### Pandas trick #3: where

In [9]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    np.random.randn(6,4),
    index=list("abcdef"),
    columns=list("ABCD")
)
df

,A,B,C,D
a,-0.436302,-0.174455,0.323573,1.848085
b,0.300921,-0.134063,-0.223338,-0.158234
c,-0.805149,-1.037277,0.255779,-1.652885
d,0.892422,-1.036361,1.236359,0.032912
e,1.631837,-0.367887,0.618237,-0.076409
f,1.559127,0.493105,0.005991,0.604944


In [10]:
df.where(df>0, -df)

,A,B,C,D
a,0.436302,0.174455,0.323573,1.848085
b,0.300921,0.134063,0.223338,0.158234
c,0.805149,1.037277,0.255779,1.652885
d,0.892422,1.036361,1.236359,0.032912
e,1.631837,0.367887,0.618237,0.076409
f,1.559127,0.493105,0.005991,0.604944


In [11]:
df.where(df>0, 0)

,A,B,C,D
a,0.000000,0.000000,0.323573,1.848085
b,0.300921,0.000000,0.000000,0.000000
c,0.000000,0.000000,0.255779,0.000000
d,0.892422,0.000000,1.236359,0.032912
e,1.631837,0.000000,0.618237,0.000000
f,1.559127,0.493105,0.005991,0.604944


### Pandas trick #4: ngroup()

In [12]:
import pandas as pd
df = pd.DataFrame({
    "a": [1,1,1,2,2,2],
    "b": [1,1,2,1,1,2]
})
df["idx"] = df.groupby(["a", "b"]).ngroup() + 1
df

,a,b,idx
0,1,1,1
1,1,1,1
2,1,2,2
3,2,1,3
4,2,1,3
5,2,2,4


### Pandas trick #5: resample & interpolate, pivot & stack

In [13]:
import pandas as pd
df = pd.DataFrame({
    "Date": ["Q1", "Q2", "Q3", "Q4", "Q1", "Q2", "Q3", "Q4"],
    "Scenario": [1,1,1,1,1,1,1,1],
    "Path": [1,1,1,1,2,2,2,2],
    "rate_1": [0.02213, 0.02867, 0.03426, 0.01678, 0.02218, 0.02572, 0.01235, 0.03215],
    "rate_2": [0.02213, 0.02867, 0.03426, 0.01678, 0.02218, 0.02572, 0.01235, 0.03215]
})
df

,Date,Scenario,Path,rate_1,rate_2
0,Q1,1,1,0.02213,0.02213
1,Q2,1,1,0.02867,0.02867
2,Q3,1,1,0.03426,0.03426
3,Q4,1,1,0.01678,0.01678
4,Q1,1,2,0.02218,0.02218
5,Q2,1,2,0.02572,0.02572
6,Q3,1,2,0.01235,0.01235
7,Q4,1,2,0.03215,0.03215


In [14]:
d = {
    "Q1": "2022-04-01",
    "Q2": "2022-07-01",
    "Q3": "2022-10-01",
    "Q4": "2023-01-01"
}
df["Date"] = df["Date"].map(d).fillna(df["Date"])
df

,Date,Scenario,Path,rate_1,rate_2
0,2022-04-01,1,1,0.02213,0.02213
1,2022-07-01,1,1,0.02867,0.02867
2,2022-10-01,1,1,0.03426,0.03426
3,2023-01-01,1,1,0.01678,0.01678
4,2022-04-01,1,2,0.02218,0.02218
5,2022-07-01,1,2,0.02572,0.02572
6,2022-10-01,1,2,0.01235,0.01235
7,2023-01-01,1,2,0.03215,0.03215


In [15]:
df["Date"] = pd.to_datetime(df["Date"]).dt.to_period("M")
(
    df.set_index("Date")
      .groupby(["Scenario", "Path"])
      .resample("M")
      .ffill()
)

Scenario  Path   rate_1   rate_2
Scenario Path Date                                     
1        1    2022-04         1     1  0.02213  0.02213
              2022-05         1     1  0.02213  0.02213
              2022-06         1     1  0.02213  0.02213
              2022-07         1     1  0.02867  0.02867
              2022-08         1     1  0.02867  0.02867
              2022-09         1     1  0.02867  0.02867
              2022-10         1     1  0.03426  0.03426
              2022-11         1     1  0.03426  0.03426
              2022-12         1     1  0.03426  0.03426
              2023-01         1     1  0.01678  0.01678
         2    2022-04         1     2  0.02218  0.02218
              2022-05         1     2  0.02218  0.02218
              2022-06         1     2  0.02218  0.02218
              2022-07         1     2  0.02572  0.02572
              2022-08         1     2  0.02572  0.02572
              2022-09         1     2  0.02572  0.02572
              2022-10         1     2  0.01235  0.01235
              2022-11         1     2  0.01235  0.01235
              2022-12         1     2  0.01235  0.01235
              2023-01         1     2  0.03215  0.03215

In [16]:
(
    df.set_index("Date")
      .groupby(["Scenario", "Path"])
      .resample("M")
      .interpolate(method="linear")
      .drop(["Scenario", "Path"], axis=1)
)

rate_1    rate_2
Scenario Path Date                       
1        1    2022-04  0.022130  0.022130
              2022-05  0.024310  0.024310
              2022-06  0.026490  0.026490
              2022-07  0.028670  0.028670
              2022-08  0.030533  0.030533
              2022-09  0.032397  0.032397
              2022-10  0.034260  0.034260
              2022-11  0.028433  0.028433
              2022-12  0.022607  0.022607
              2023-01  0.016780  0.016780
         2    2022-04  0.022180  0.022180
              2022-05  0.023360  0.023360
              2022-06  0.024540  0.024540
              2022-07  0.025720  0.025720
              2022-08  0.021263  0.021263
              2022-09  0.016807  0.016807
              2022-10  0.012350  0.012350
              2022-11  0.018950  0.018950
              2022-12  0.025550  0.025550
              2023-01  0.032150  0.032150

In [17]:
df = (
    df.set_index("Date")
      .groupby(["Scenario", "Path"])
      .resample("M")
      .asfreq()
      .drop(["Scenario", "Path"], axis=1)
      .interpolate(method="linear")
)
df

rate_1    rate_2
Scenario Path Date                       
1        1    2022-04  0.022130  0.022130
              2022-05  0.024310  0.024310
              2022-06  0.026490  0.026490
              2022-07  0.028670  0.028670
              2022-08  0.030533  0.030533
              2022-09  0.032397  0.032397
              2022-10  0.034260  0.034260
              2022-11  0.028433  0.028433
              2022-12  0.022607  0.022607
              2023-01  0.016780  0.016780
         2    2022-04  0.022180  0.022180
              2022-05  0.023360  0.023360
              2022-06  0.024540  0.024540
              2022-07  0.025720  0.025720
              2022-08  0.021263  0.021263
              2022-09  0.016807  0.016807
              2022-10  0.012350  0.012350
              2022-11  0.018950  0.018950
              2022-12  0.025550  0.025550
              2023-01  0.032150  0.032150

In [18]:
cols = df.columns
df = (
    df.reset_index()
      .pivot(index=["Scenario", "Path"], columns="Date", values=cols)
      .stack(level=0)
      .reset_index()
)
df

Date,Scenario,Path,level_2,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
0,1,1,rate_1,0.02213,0.02431,0.02649,0.02867,0.030533,0.032397,0.03426,0.028433,0.022607,0.01678
1,1,1,rate_2,0.02213,0.02431,0.02649,0.02867,0.030533,0.032397,0.03426,0.028433,0.022607,0.01678
2,1,2,rate_1,0.02218,0.02336,0.02454,0.02572,0.021263,0.016807,0.01235,0.018950,0.025550,0.03215
3,1,2,rate_2,0.02218,0.02336,0.02454,0.02572,0.021263,0.016807,0.01235,0.018950,0.025550,0.03215


### Pandas trick #6: mapping

In [19]:
countries = pd.Series([
    'United States',
    'Canada',
    'Mexico',
    'Belgium',
    'United Kingdom',
    'Thailand'
])
groups = {
    'North America': ('United States', 'Canada', 'Mexico', 'Greenland'),
    'Europe': ('France', 'Germany', 'United Kingdom', 'Belgium')
}

In [20]:
groups = {x: k for k, v in groups.items() for x in v}
groups

{'United States': 'North America',
 'Canada': 'North America',
 'Mexico': 'North America',
 'Greenland': 'North America',
 'France': 'Europe',
 'Germany': 'Europe',
 'United Kingdom': 'Europe',
 'Belgium': 'Europe'}

In [21]:
countries.map(groups).fillna(countries)

0    North America
1    North America
2    North America
3           Europe
4           Europe
5         Thailand
dtype: object

In [22]:
countries.map(groups).fillna(-999)

0    North America
1    North America
2    North America
3           Europe
4           Europe
5             -999
dtype: object

### Pandas trick #7: stack and unstack

In [23]:
import pandas as pd
import numpy as np

tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

A         B
first second                    
bar   one     0.818260  0.319464
      two     0.031095 -1.297223
baz   one    -0.966224 -2.437250
      two     0.485767 -1.557461
foo   one    -0.446143  0.936670
      two    -0.826826  0.425988
qux   one    -0.663906  1.873612
      two     1.154674  0.132909

In [24]:
stacked = df.stack()
stacked

first  second   
bar    one     A    0.818260
               B    0.319464
       two     A    0.031095
               B   -1.297223
baz    one     A   -0.966224
               B   -2.437250
       two     A    0.485767
               B   -1.557461
foo    one     A   -0.446143
               B    0.936670
       two     A   -0.826826
               B    0.425988
qux    one     A   -0.663906
               B    1.873612
       two     A    1.154674
               B    0.132909
dtype: float64

In [25]:
stacked.unstack()

A         B
first second                    
bar   one     0.818260  0.319464
      two     0.031095 -1.297223
baz   one    -0.966224 -2.437250
      two     0.485767 -1.557461
foo   one    -0.446143  0.936670
      two    -0.826826  0.425988
qux   one    -0.663906  1.873612
      two     1.154674  0.132909

### Pandas trick #8: multi-index, slice

In [26]:
import pandas as pd
import numpy as np

def mklbl(prefix, n):
    return ["%s%s" % (prefix, i) for i in range(n)]

miindex = pd.MultiIndex.from_product(
    [mklbl("A", 4), mklbl("B", 2), mklbl("C", 4), mklbl("D", 2)]
)

micolumns = pd.MultiIndex.from_tuples(
    [("a", "foo"), ("a", "bar"), ("b", "foo"), ("b", "bah")], names=["lvl0", "lvl1"]
)

dfmi = (
    pd.DataFrame(
        np.arange(len(miindex) * len(micolumns)).reshape(
            (len(miindex), len(micolumns))
        ),
        index=miindex,
        columns=micolumns,
    )
    .sort_index()
    .sort_index(axis=1)
)
dfmi

lvl0           a         b     
lvl1         bar  foo  bah  foo
A0 B0 C0 D0    1    0    3    2
         D1    5    4    7    6
      C1 D0    9    8   11   10
         D1   13   12   15   14
      C2 D0   17   16   19   18
...          ...  ...  ...  ...
A3 B1 C1 D1  237  236  239  238
      C2 D0  241  240  243  242
         D1  245  244  247  246
      C3 D0  249  248  251  250
         D1  253  252  255  254

[64 rows x 4 columns]

In [27]:
dfmi.loc[(slice("A1", "A3"), slice(None), slice("C1", "C3"))]

lvl0           a         b     
lvl1         bar  foo  bah  foo
A1 B0 C1 D0   73   72   75   74
         D1   77   76   79   78
      C2 D0   81   80   83   82
         D1   85   84   87   86
      C3 D0   89   88   91   90
         D1   93   92   95   94
   B1 C1 D0  105  104  107  106
         D1  109  108  111  110
      C2 D0  113  112  115  114
         D1  117  116  119  118
      C3 D0  121  120  123  122
         D1  125  124  127  126
A2 B0 C1 D0  137  136  139  138
         D1  141  140  143  142
      C2 D0  145  144  147  146
         D1  149  148  151  150
      C3 D0  153  152  155  154
         D1  157  156  159  158
   B1 C1 D0  169  168  171  170
         D1  173  172  175  174
      C2 D0  177  176  179  178
         D1  181  180  183  182
      C3 D0  185  184  187  186
         D1  189  188  191  190
A3 B0 C1 D0  201  200  203  202
         D1  205  204  207  206
      C2 D0  209  208  211  210
         D1  213  212  215  214
      C3 D0  217  216  219  218
         D1  221  220  223  222
   B1 C1 D0  233  232  235  234
         D1  237  236  239  238
      C2 D0  241  240  243  242
         D1  245  244  247  246
      C3 D0  249  248  251  250
         D1  253  252  255  254

In [28]:
dfmi.loc[("A0", "B0", slice(None), "D1")]

lvl0          a       b    
lvl1        bar foo bah foo
A0 B0 C0 D1   5   4   7   6
      C1 D1  13  12  15  14
      C2 D1  21  20  23  22
      C3 D1  29  28  31  30

### Pandas trick #9: time series  

In [29]:
import pandas as pd
import datetime

start_date = datetime.datetime.strptime("2023-01", "%Y-%m")
date_generated = pd.date_range(start_date, periods=12, freq="3M")
qtr = ["Q" + str(i) for i in range(1, 12)]
d = dict(zip(qtr, date_generated.strftime("%Y-%m")))
d

{'Q1': '2023-01',
 'Q2': '2023-04',
 'Q3': '2023-07',
 'Q4': '2023-10',
 'Q5': '2024-01',
 'Q6': '2024-04',
 'Q7': '2024-07',
 'Q8': '2024-10',
 'Q9': '2025-01',
 'Q10': '2025-04',
 'Q11': '2025-07'}

### Pandas trick #10: create dummy variables from a list

In [30]:
d = {
    "col1": ["A", "B", "C"],
    "col2": [33, 2.5, 42],
    "col3": [["Apple", "Orange", "Banana"], ["Apple", "Grape"], ["Banana"]]
}

import pandas as pd
df = pd.DataFrame(d)
df.pop("col3").str.join("|").str.get_dummies()

,Apple,Banana,Grape,Orange
0,1,1,0,1
1,1,0,1,0
2,0,1,0,0


In [31]:
d = {
    "col1": ["A", "B", "C"],
    "col2": [33, 2.5, 42],
    "col3": [["Apple", "Orange", "Banana"], ["Apple", "Grape"], ["Banana"]]
}

import pandas as pd
df = pd.DataFrame(d)
df.join(df.pop("col3").str.join("|").str.get_dummies())

,col1,col2,Apple,Banana,Grape,Orange
0,A,33.0,1,1,0,1
1,B,2.5,1,0,1,0
2,C,42.0,0,1,0,0


In [32]:
# use MultiLableBinarizer
d = {
    "col1": ["A", "B", "C"],
    "col2": [33, 2.5, 42],
    "col3": [["Apple", "Orange", "Banana"], ["Apple", "Grape"], ["Banana"]]
}

import pandas as pd
df = pd.DataFrame(d)

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=True)
df.join(pd.DataFrame.sparse.from_spmatrix(mlb.fit_transform(df.pop("col3")), index=df.index, columns=mlb.classes_))


,col1,col2,Apple,Banana,Grape,Orange
0,A,33.0,1,1,0,1
1,B,2.5,1,0,1,0
2,C,42.0,0,1,0,0


### Pandas tricks #11: value counts to percentage

In [33]:
import pandas as pd

df = pd.DataFrame({
    "A": ["foo", "foo", "foo", "foo", "foo", "bar", "bar", "bar", "bar"],
    "B": ["one", "one", "one", "two", "two", "one", "one", "two", "two"],
    "C": ["small", "large", "large", "small", "small", "large", "small", "small", "large"],
    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]
})
print(df["A"].value_counts())
print(df["A"].value_counts(normalize=True))

foo    5
bar    4
Name: A, dtype: int64
foo    0.555556
bar    0.444444
Name: A, dtype: float64


### Pandas tricks #12: explode() Transform each element of a list-like to a row, replicating index values

In [34]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'A': [[0, 1, 2], 'foo', [], [3, 4]],
                   'B': 1,
                   'C': [['a', 'b', 'c'], np.nan, [], ['d', 'e']]})
df

,A,B,C
0,"[0, 1, 2]",1,"[a, b, c]"
1,foo,1,NaN
2,[],1,[]
3,"[3, 4]",1,"[d, e]"


In [35]:
df.explode("A")

,A,B,C
0,0,1,"[a, b, c]"
0,1,1,"[a, b, c]"
0,2,1,"[a, b, c]"
1,foo,1,NaN
2,NaN,1,[]
3,3,1,"[d, e]"
3,4,1,"[d, e]"


In [36]:
df.explode(["A", "C"])

,A,B,C
0,0,1,a
0,1,1,b
0,2,1,c
1,foo,1,NaN
2,NaN,1,NaN
3,3,1,d
3,4,1,e
